# Bounds

## Input 
### A - numpy array, Adjacency Matrix
### X - numpy array, Node Features
### train_ids - numpy array, Train Node IDs. Make sure that Train ids range [0, V-1]
### nrounds_smooth - No of rounds of Smoothing. Use nrounds_smooth = 0 to use only raw node features. 
### L - No. of Layers of MLP. If Linear classifier use L = 1
### K - No of Classes. K = 2 for Binary classification
### W - list of parameters (numpy 2D arrays). Each element of the list is parameter of a layer of MLP. These parameters needn't be ordered. Just pass each layer parameter as an element of the list

In [1]:
import numpy as np

In [2]:
def compute_bounds(N, X, distX, train_ids, L, K, W):
    
    # Define constants in bounds
    ## Lipchitz parameter c - Not sure how to measure. Setting it to 1/max(em). Look at eq following eq.14 in the paper
    c = 1/max(distX.keys())
    ## N_o = No of Train nodes 
    N0 = train_ids.shape[0]
    ## alpha - There exists such a value! See Assumption 3 and discussion after Equation 15, 
    ## it governs probability of undesirable event. So should be set to high value i.e., 1/4
    alpha = 0.25 - 1e-6
    ## delta - Confidence, setting it 0.9
    delta = 0.9
    ## gamma - Margin setting it low value 1e-4
    gamma = 1e-4
    ## b - Apparently from a spectral norm inequality. I couldn't locate it in the reference. 
    ## They didn't even point the equation out!!
    b = 1
    ## C - Upper bound on Frenobius Norm of Wieghts "W_Fn". Setting it to ceil() of the max Frenobius norm, W_Fn
    W_Fn = np.sum([np.sum(np.square(w)) for w in W])**0.5
    C = np.ceil(W_Fn)
    ## Norm of the Smoothed Features
    B = np.linalg.norm(X, axis=1)
    B0 = np.max(B[train_ids])
    
    def compute_upper_bound(vi, dxi):
        ub = c * K * dxi
        ub += ((W_Fn*b) * (dxi**2) /(N0**alpha*(gamma/8)**2))
        ub += N0**(2*alpha-1)
        xb = (L*C*2*B[vi])/(delta*gamma)
        ub += N0**(-2*alpha) * np.log(xb + 1e-6)
        return ub
    
    gen_bounds = {}
    test_ids = np.setdiff1d(np.arange(N), train_ids)
    for vi in test_ids:
        gen_bounds[vi] = compute_upper_bound(vi, distX[vi])
    maxCS = max([gen_bounds[x] for x in gen_bounds])
    gen_bounds = {x: gen_bounds[x]/maxCS for x in gen_bounds}
    return gen_bounds
    

In [3]:
import torch
import torch.nn.functional as F

In [4]:
from ogb.nodeproppred import PygNodePropPredDataset

Using backend: pytorch[07:24:00] /opt/dgl/src/runtime/tensordispatch.cc:
43: TensorDispatcher: dlopen failed: /home/shreyshs/anaconda3/lib/python3.9/site-packages/dgl/tensoradapter/pytorch/libtensoradapter_pytorch_1.10.2.so: cannot open shared object file: No such file or directory


In [5]:
from outcome_correlation import process_adj

In [6]:
import pickle

In [7]:
import os

In [8]:
import hnswlib

In [9]:
def get_dataset_bounds(dataset_name, model_name):
    
    dataset = PygNodePropPredDataset(name='ogbn-{}'.format(dataset_name))
    data = dataset[0]
    adj, D_isqrt = process_adj(data) # not really required
    adj = adj.to_symmetric()
    split_idx = dataset.get_idx_split()
    
    #train_ids = np.concatenate([split_idx ['train'], split_idx['valid']])
    train_ids = split_idx ['train']
    X = data.x
    embeddings = None 
    if dataset_name == 'products':
        embeddings = torch.load('embeddings/spectral{}.pt'.format(dataset_name))
    else:
        embeddings = torch.load('embeddings/diffusion{}.pt'.format(dataset_name))
    if dataset_name == 'arxiv':
        embeddings = torch.cat([embeddings, torch.load('embeddings/spectral{}.pt'.format(dataset_name))], dim=-1)
    if model_name == 'linear' or model_name == 'mlp':
        X = torch.cat([X, embeddings], dim=-1)
    X = X.numpy()
    K = dataset.num_classes
    N = data.num_nodes
    
    def get_ann_distance(Y):
        p = hnswlib.Index(space = 'l2', dim = Y.shape[1])
        p.init_index(max_elements = Y.shape[0]+1, ef_construction = int(0.05*Y.shape[0]), M = 256)
        p.add_items(Y[train_ids], train_ids.numpy())
        labels, distances = p.knn_query(Y, k = 5)
        agg_distance = {}
        for i in range(N):
            agg_distance[i] = distances[i, 0]
        return agg_distance
    
    # Compute distance to the Train nodes
    def get_agg_distance(Y):
        agg_distance = {}
        for i in range(A.shape[0]):
            agg_distance[i] = float('inf')
            for j in train_ids:
                agg_distance[i] = min(agg_distance[i],
                                      np.linalg.norm(Y[i] - Y[j]))
        return agg_distance
    
    distX = None
    if dataset_name == 'cora' or dataset_name == 'citeseer' or dataset_name == 'pubmed':
        distX = get_agg_distance(X)
    else:
        distX = get_ann_distance(X)
    
    save_path = 'generalization_bounds/{}-{}'.format(dataset_name, model_name)
    if not os.path.isdir(save_path):
        os.mkdir(save_path)
    
    for runs in range(10):
    
        model_weights = torch.load('models/{}_{}/model-{}.tar'.format(dataset_name, model_name, runs))['model_state_dict']
        
        a = None 
        b = None
        W = []
        
        if model_name == 'mlp':
            for i in range(3):
                a = model_weights['lins.{}.weight'.format(i)].numpy()
                b = model_weights['lins.{}.bias'.format(i)].numpy()
                b = b.reshape((b.shape[0], 1))
                W.append(np.hstack((a, b)))
        else:
            a = model_weights['lin.weight'].numpy()
            b = model_weights['lin.bias'].numpy()
            b = b.reshape((b.shape[0], 1))
            W.append(np.hstack((a, b)))

        L = len(W)
        gen_bounds = compute_bounds(N, X, distX, train_ids, L, K, W)
        y_score = torch.load('models/{}_{}/{}.pt'.format(dataset_name, model_name, runs), map_location='cpu')
        yhat = y_score.argmax(dim=-1, keepdim=True).numpy()
        
        E_matrix = np.zeros((N, K))
        for i in gen_bounds:
            E_matrix[i] = E_matrix[i] + gen_bounds[i]
        E_matrix = torch.tensor(E_matrix, dtype=torch.float32)
        torch.save(E_matrix, '{}/fixed-{}.th'.format(save_path, runs))
        
        E_matrix = np.zeros((N, K))
        for i in gen_bounds:
            E_matrix[i] = E_matrix[i] + (1 - gen_bounds[i])/K
            pred_indx = int(yhat[i])
            E_matrix[i, pred_indx] = gen_bounds[i]
        E_matrix = torch.tensor(E_matrix, dtype=torch.float32)
        torch.save(E_matrix, '{}/spread-{}.th'.format(save_path, runs))
        
        E_matrix = np.zeros((N, K))
        for i in gen_bounds:
            rc = np.random.random(K)
            rc /= rc.sum()
            E_matrix[i] = E_matrix[i] + rc
        E_matrix = torch.tensor(E_matrix, dtype=torch.float32)
        torch.save(E_matrix, '{}/random-{}.th'.format(save_path, runs))
        

In [9]:
get_dataset_bounds('cora', 'plain')

In [10]:
get_dataset_bounds('cora', 'linear')

In [11]:
get_dataset_bounds('cora', 'mlp')

In [22]:
get_dataset_bounds('citeseer', 'plain')

In [13]:
get_dataset_bounds('citeseer', 'linear')

In [14]:
get_dataset_bounds('citeseer', 'mlp')

In [15]:
get_dataset_bounds('pubmed', 'plain')

In [16]:
get_dataset_bounds('pubmed', 'linear')

In [18]:
get_dataset_bounds('pubmed', 'mlp')

In [10]:
get_dataset_bounds('arxiv', 'plain')

In [11]:
get_dataset_bounds('arxiv', 'linear')

In [12]:
get_dataset_bounds('arxiv', 'mlp')

In [10]:
get_dataset_bounds('products', 'plain')

In [11]:
get_dataset_bounds('products', 'linear')

In [12]:
get_dataset_bounds('products', 'mlp')